# Introducing transfer entropy as a measure of information flow

DEFinition of conditional entropy:
\begin{equation}
H(Y \mid X) = - \sum_j p(y_j \mid x_i) \ln p(y_j \mid x_i)
\end{equation}

DEFinition of Transfer Entropy (TE) à la Schreiber:
\begin{equation}
T_{X\rightarrow Y} = H\left( Y_t \mid Y_{t-1:t-L}\right) - H\left( Y_t \mid Y_{t-1:t-L}, X_{t-1:t-L}\right),
\end{equation}

## Example: Biased Rock-Paper-Scissors

We observe Andy, Bob and Claire playing the game of "Rock-Paper-Scissors". In order to examine the information flow-based measure of TE, we assume that the three players have different playing strategies, which may or may not depend on information received from other players.

Andy ($A$) plays randomly, while Bob ($B$) is trying to read Andy, which is successful with a probability of $x$, otherwise Bob also plays randomly. Claire ($C$), on other hand, reads both Andy and Bob: she reads Andy with a probability of $y$, while she can detect when Bob is successful in reading Andy, therefore choosing a hand that is drawing the round. 

### Question:

Each round, all three play their hand ($a_t, b_t, c_t$). We, the observers, can only read out, who is winning each round as a function of all the variables $f(a_t, b_t, c_t)$. How much information can we extract from this readout alone?

In [112]:
class RPS():
    def __init__(self, _players = []):
        self.rock = 0
        self.paper = 1
        self.scissors = 2
        self.allhands = [rock, paper, scissors]
        self.item = ["rock", "paper", "scissors"]
        self.players = _players
        self.nwins = [0]*len(_players)

    def win(self, args):
        if len(args) > 2:
            winners = []
            for arg in args:
                for arg2 in args:
                    if arg == arg2:
                        pass
                    else:
                        winners.append(win([arg, arg2]))
            if len(list(set(winners))) == 1:
                return winners[0]
            else:
                return []
        elif len(args) == 2:
            return args[0] if args[1]-args[0] == 0 else (0 if sum(args) == 2 else (1 if sum(args) == 1 else 2))
        elif len(args) == 1:
            return (args[0]+1)%3
        else:
            print("Wrong number of arguments (at least 1).")
            return -1
    
    def getAllWinners(self, winning_hand, playing_hands):
        winners = []
        idx = 0
        while True: 
            try:
                winners.append(playing_hands.index(winning_hand,idx))
                idx = winners[-1] + 1
            except ValueError:
                return winners

    def names(self):
        return self.players
            
    def nps(self):
        return len(self.players)
    
    def up(self, ind):
        self.nwins[ind] += 1
        
    def play(self, hands=[]):
        if len(hands) == 0:
            pass
        else:
            if len(hands) == self.nps():
                allWinners = getAllWinners(self.win(hands), hands)
                #print("Hands:", [self.item[hand] for hand in hands])
                #print("Winners:", [self.names()[each] for each in allWinners])
                for each in allWinners:
                    self.up(each)
                #print(self.nwins)
            else:
                print("Number of hands does not match number of players")
        
        

In [143]:
import numpy as np

class PlayerEngine():
    def __init__(self):
        self.obs = []
        self.nextplay = np.random.choice([0,1,2])
        self.thisplay = -1
        
    def set_observable(self, _obs):
        self.obs.append(_obs)
        
    def get_win(self, hand_to_beat):
        return (hand_to_beat+1)%3
        
    def play_random(self, printit=False):
        self.thisplay = self.nextplay
        if printit:
            print(self.thisplay)
        self.nextplay = np.random.choice([0,1,2])
        return self.thisplay
    
    def play_semi(self, _prob):
        rp = np.random.rand()
        self.thisplay = self.nextplay
        if rp < _prob:
            if len(self.obs) > 0:
                to_beat = self.obs[0].nextplay
                self.nextplay = self.get_win(to_beat)
                return self.thisplay
            else:
                return self.play_random()
        else:
            return self.play_random()
            

In [145]:
pnames = ["Andy", "Bob", "Claire"]
myRPS = RPS(_players=pnames)
plays = [PlayerEngine(), PlayerEngine(), PlayerEngine()]
plays[1].set_observable(plays[0])
nrounds = 10000

for iround in range(nrounds):
    this_round = [plays[0].play_random(), plays[1].play_semi(.5), plays[2].play_random()]
    #print(this_round)
    myRPS.play(this_round)
    #print("")
print(myRPS.nwins)


[1692, 4986, 3378]
